In [1]:
import requests

# Datos necesarios para la autenticación
client_id = "a5df46ce109348498ac463cd83a3465d"  # Reemplaza con tu client ID
client_secret = "5f1ba3e73cb84469bfc1d9f618f09434"  # Reemplaza con tu client secret

# URL del endpoint de Spotify para obtener el token
auth_url = "https://accounts.spotify.com/api/token"

# Encabezados y datos de la solicitud
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

# Hacer la solicitud POST
response = requests.post(auth_url, headers=headers, data=data)

# Verificar el estado y obtener el token
if response.status_code == 200:
    token = response.json().get("access_token")
    print(f"Token obtenido: {token}")
else:
    print(f"Error al obtener el token: {response.status_code}")
    print(response.json())


Token obtenido: BQDRZ98sP8XNSS8cRccOSV2DDTs0LIeQpaamlDHZ2PGj-zSsmNrFek4nOZFucnH7Lzwyzrq6zG2SM6iQzAAHmlMhrg0Ebl-cxRgvPGgBtdMoG1SRPV7hJzI9FTcb8_3PBSiyaeN2YKM


In [2]:
# ID de la playlist
# https://open.spotify.com/playlist/7uXP8jfO3aOCIrAYnMRg1t?si=27a64f61132b4dc4
playlist_id = "7uXP8jfO3aOCIrAYnMRg1t"

# URL de la API para obtener detalles de la playlist
url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

# Encabezado con el token de autenticación
headers = {
    "Authorization": f"Bearer {token}"
}

# Hacer la solicitud
response = requests.get(url, headers=headers)

# Verificar la respuesta
if response.status_code == 200:
    playlist_data = response.json()
    print("Nombre de la playlist:", playlist_data["name"])
    print("Descripción:", playlist_data["description"])
    print("Número de pistas:", playlist_data["tracks"]["total"])
    print("Seguidores:", playlist_data["followers"]["total"])
else:
    print(f"Error al obtener la playlist: {response.status_code}")
    print(response.json())



Nombre de la playlist: promusicae semana 49
Descripción: 
Número de pistas: 100
Seguidores: 0


In [ ]:
import pandas as pd
import time
tracks = playlist_data["tracks"]["items"]

# Crear una lista para almacenar la información de cada track
track_list = []
headers = {
    "Authorization": f"Bearer {token}"
}
# Recorrer cada track para obtener los datos relevantes
for track in tracks:
    # Extraer ID y nombre del primer artista
    first_artist = track["track"]["artists"][0]  # Tomamos solo el primer artista
    artist_id = first_artist["id"]
    artist_name = first_artist["name"]

    # Obtener los géneros del artista consultando la API
    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    artist_response = requests.get(artist_url, headers=headers)
    
    # Si la consulta fue exitosa, extraer los géneros
    if artist_response.status_code == 200:
        artist_data = artist_response.json()
        genres = ", ".join(artist_data.get("genres", ["Desconocido"]))
    else:
        genres = "Desconocido"  # Si hay error, dejar un valor por defecto
    
    # Construir el diccionario con la info del track
    track_info = {
        "track_name": track["track"]["name"],
        "artist_name": ", ".join([artist["name"] for artist in track["track"]["artists"]]),
        "album_name": track["track"]["album"]["name"],
        "duration_s": track["track"]["duration_ms"] / 1000,
        "release_date": track["track"]["album"]["release_date"],
        "popularity": track["track"]["popularity"],
        "explicit": track["track"]["explicit"],
        "track_id": track["track"]["id"],
        "artist_id": artist_id,
        "genres": genres
    }
    
    track_list.append(track_info)

    # Pequeña pausa para evitar el límite de la API
    time.sleep(0.5)

# Convertir la lista a un DataFrame de pandas
df_tracks_g = pd.DataFrame(track_list)



In [ ]:
import pandas as pd
import time
tracks = playlist_data["tracks"]["items"]

# Crear una lista para almacenar la información de cada track
track_list = []
headers = {
    "Authorization": f"Bearer {token}"
}
# Recorrer cada track para obtener los datos relevantes
for track in tracks:
    # Extraer ID y nombre del primer artista
    first_artist = track["track"]["artists"][0]  # Tomamos solo el primer artista
    artist_id = first_artist["id"]
    artist_name = first_artist["name"]

    # Obtener artista consultando la API
    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    artist_response = requests.get(artist_url, headers=headers)
    
    # Si la consulta fue exitosa, extraer los followers
    if artist_response.status_code == 200:
        artist_data = artist_response.json()
        popularity = artist_data.get("popularity", 0)  # Popularidad del artista
        followers = artist_data.get("followers", {}).get("total", 0)  # Seguidores
    else:
        popularity = 0
        followers = 0  # Si hay error, dejar un valor por defecto
    
    # Construir el diccionario con la info del track
    track_info = {
        "track_name": track["track"]["name"],
        "artist_name": ", ".join([artist["name"] for artist in track["track"]["artists"]]),
        "album_name": track["track"]["album"]["name"],
        "duration_s": track["track"]["duration_ms"] / 1000,
        "release_date": track["track"]["album"]["release_date"],
        "popularity": track["track"]["popularity"],  # Popularidad del track
        "explicit": track["track"]["explicit"],
        "track_id": track["track"]["id"],
        "artist_id": artist_id,
        "artist_popularity": popularity,  # Nueva columna: Popularidad del artista
        "followers": followers  # Nueva columna: Seguidores del artista
    }
    
    track_list.append(track_info)

    # Pequeña pausa para evitar el límite de la API
    time.sleep(0.5)

# Convertir la lista a un DataFrame de pandas
df_tracks = pd.DataFrame(track_list)



In [11]:
df_tracks[df_tracks['artist_name'].str.count(',') > 1]

,track_name,artist_name,album_name,duration_s,release_date,popularity,explicit,track_id,artist_id,artist_popularity,followers
27,Ma Meilleure Ennemie (from the series Arcane L...,"Stromae, Pomme, Arcane",Ma Meilleure Ennemie (from the series Arcane L...,147.973,2024-11-23,83,False,4lriIG2vNqwDWzOj2I9rtj,5j4HeCoUlzhfWtjAfM1acR,77,3718481
28,AMANECIÓ,"Quevedo, De La Rose, De La Ghetto",BUENAS NOCHES,255.333,2024-11-22,72,False,4DxbTPS1DUzTKzpNfCbYjy,52iwsT98xCoGgiGntTiR7K,83,5784587
29,+57,"KAROL G, Feid, DFZM, Ovy On The Drums, J Balvi...",+57,298.130,2024-11-08,84,True,3mg9jYNC1NihbthlglzrWP,790FomKkXshlbRYZFtlgla,89,55921859
41,ESTOY PUTIAO,"Feid, Sky Rompiendo, Dei V",LOS 9 DE FERXXO Y SKY ROMPIENDO,223.039,2024-11-29,72,True,0lhcaiGyAkmZg5RaC99ZbH,2LRoIwlKmHjgvigdNGBHNo,90,16810752
45,Santa,"Rvssian, Rauw Alejandro, Ayra Starr",Santa,193.038,2024-04-04,77,False,5bi0gh89wRuH2OgjdAKFsb,1fctva4kpRbg2k3v7kwRuS,73,2525732
47,Hm Hm Hm - Remix,"Rvfv, Youka, Wesh Music",Hm Hm Hm (Remix),181.052,2024-11-29,67,False,53OKRnaKKAT3bCkF8WIY8C,2CCgb0KApjfQDuTppovpf8,74,865888
53,DEPORTIVO,"Blessd, Anuel AA, Ovy On The Drums",DEPORTIVO,218.701,2024-11-14,85,True,0fKK51bU6lcCCwdNnv64t3,1TA5sGRlKUJXBN4ZyJuDIX,83,6978343
70,BADGYAL,"SAIKO, JC Reyes, Dei V",SAKURA,253.962,2024-04-26,73,True,3ng8tfwvzR4BBwa9yaMms6,2O8vbr4RYPpk6MRA4fio7u,74,1651816
76,Lo Que Tiene,"Morad, Beny Jr, Rvfv",Lo Que Tiene,259.200,2024-03-08,76,False,6RqD0fpAaKEZvQd0QrTj7j,4az97MtWmBQ5Db3GfDh9j9,81,4060399
87,Vagabundo,"Sebastian Yatra, Manuel Turizo, Beéle",Vagabundo,215.600,2023-05-12,81,False,1MB8kTH7VKvAMfL9SHgJmG,07YUOmWljBTXwIseAUd9TW,81,23806527


In [9]:
# vamos a sacar los datos del segundo artista
import pandas as pd
import requests
import time

# Asumiendo que ya tienes `playlist_data`, `token`, y la lista `tracks`
second_artist_data = []
headers = {
    "Authorization": f"Bearer {token}"
}

for idx, track in enumerate(tracks):
    artists = track["track"]["artists"]
    
    if len(artists) > 1:
        second_artist = artists[1]
        second_artist_name = second_artist["name"]
        second_artist_id = second_artist["id"]

        # Consulta a la API para obtener seguidores
        artist_url = f"https://api.spotify.com/v1/artists/{second_artist_id}"
        response = requests.get(artist_url, headers=headers)
        
        if response.status_code == 200:
            artist_data = response.json()
            followers = artist_data.get("followers", {}).get("total", None)
        else:
            followers = None
    else:
        second_artist_name = None
        followers = None

    second_artist_data.append({
        "index": idx,
        "second_artist_name": second_artist_name,
        "second_artist_followers": followers
    })

    time.sleep(0.5)  # Para no saturar la API

# Crear el nuevo DataFrame con los datos del segundo artista
df_second_artist = pd.DataFrame(second_artist_data).set_index("index")


In [16]:
df_second_artist.reset_index().drop('index', axis=1).to_csv('./second_artist.csv', sep=';', index=False, encoding='utf-8')

In [6]:
df_tracks.to_csv('./tracks_w_artist.csv', sep=';', index=False, encoding='utf-8')

> Usamos YT para sacar los parámetros del módulo librosa

In [ ]:
# Analizar audio con librosa
import librosa
def analyze_audio(audio_file):
    """
    Extrae características clave del audio usando librosa.
    """
    y, sr = librosa.load(audio_file)

    # Duración
    duration = librosa.get_duration(y=y, sr=sr)

    # Tempo (BPM)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

    # Key (escala musical)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    key = chroma.mean(axis=1).argmax()

    # RMS
    rms = librosa.feature.rms(y=y)
    avg_rms = np.mean(rms)

    # Flatness
    flatness = librosa.feature.spectral_flatness(y=y)
    avg_flatness = flatness.mean()

    # Onsets
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)
    num_onsets = len(onset_times)
    onset_density = num_onsets / duration

    # Retornar resultados en un diccionario
    return {
        "duration": duration,
        "tempo": float(tempo[0]).__round__(2),
        "key": int(key),
        "avg_rms": float(avg_rms),
        "avg_flatness": float(avg_flatness),
        "num_onsets": num_onsets,
        "onset_density": onset_density,
    }

In [ ]:
tracks_df = pd.read_excel("tracks_prod.xlsx", index_col=0, usecols=[1,2]).reset_index() # excel al que le añadimos manualmente la productora del tema

In [71]:
tracks_df

,track_name,Productora
0,GRAN VÍA,DQE PRODUCTIONS
1,SHIBATTO,DQE PRODUCTIONS
2,Si Antes Te Hubiera Conocido,UNIVERSAL
3,KASSANDRA,DQE PRODUCTIONS
4,DEGENERE 👑 (feat. benny blanco),ONE WORLD MUSIC
...,...,...
95,Pégate,SONY MUSIC
96,Revolú,SONY MUSIC
97,Orgullosa De Papá,RABAT MUSIC SL
98,Columbia,TASTE THE FLOOR RECORDS


In [72]:
df_tracks = df_tracks.merge(tracks_df['Productora'], left_index=True, right_index=True)

In [73]:
for i in range(100):
    df_tracks.loc[i, 'colab'] = ', ' in df_tracks.loc[i, 'artist_name']

In [ ]:
# Crear DataFrame para guardar los atributos
audio_features = []
import os
import pandas as pd
import librosa
import numpy as np
import subprocess

audio_folder = "audios"
for index, row in tracks_df.iterrows():
    audio_path = os.path.join(audio_folder, f"{index}.mp3")
    if os.path.exists(audio_path):
        print(f"Analizando audio: {audio_path}")
        try:
            features = analyze_audio(f"{audio_path}")
            audio_features.append(features)
        except Exception as e:
            print(f"Error al analizar {audio_path}: {e}")

# Crear DataFrame con los atributos y guardar a Excel
features_df = pd.DataFrame(audio_features)


Analizando audio: audios\0.mp3
Analizando audio: audios\1.mp3
Analizando audio: audios\2.mp3
Analizando audio: audios\3.mp3
Analizando audio: audios\4.mp3
Analizando audio: audios\5.mp3
Analizando audio: audios\6.mp3
Analizando audio: audios\7.mp3
Analizando audio: audios\8.mp3
Analizando audio: audios\9.mp3
Analizando audio: audios\10.mp3
Analizando audio: audios\11.mp3
Analizando audio: audios\12.mp3
Analizando audio: audios\13.mp3
Analizando audio: audios\14.mp3
Analizando audio: audios\15.mp3
Analizando audio: audios\16.mp3
Analizando audio: audios\17.mp3
Analizando audio: audios\18.mp3
Analizando audio: audios\19.mp3
Analizando audio: audios\20.mp3
Analizando audio: audios\21.mp3
Analizando audio: audios\22.mp3
Analizando audio: audios\23.mp3
Analizando audio: audios\24.mp3
Analizando audio: audios\25.mp3
Analizando audio: audios\26.mp3
Analizando audio: audios\27.mp3
Analizando audio: audios\28.mp3
Analizando audio: audios\29.mp3
Analizando audio: audios\30.mp3
Analizando audio: 

In [74]:
df_tracks = df_tracks.merge(features_df, left_index=True, right_index=True)

In [ ]:
df_tracks.to_csv('./tracks.csv')